In [0]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import nltk
import string
import re
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from keras.models import load_model

In [63]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
data1 = pd.read_csv('train_2kmZucJ.csv')
data1

In [0]:
data2 = pd.read_csv('test_oJQbWVk.csv')
data2

In [68]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')
#stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def data_cleaning(tweet):
  tweet = re.sub(r'^https?:\/\/.*[\r\n]*', '', tweet, flags=re.MULTILINE)
  tokens = tweet.split()
  table = str.maketrans('', '', string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  #stop_words = set(stopwords.words('english'))
  tokens = [w for w in tokens if not w in stop_words]
  tokens = [word for word in tokens if len(word) > 1]
  tokens = [word.lower() for word in tokens]
  #tokens = [porter.stem(word) for word in tokens]
  return tokens

In [0]:
def data_preparation(data1):
  for i in range(len(data1)):
    str1 = " "
    token = data_cleaning(data1['tweet'][i])
    data1['tweet'][i] = str1.join(token)
  return data1

In [0]:
def data_split(data1):
  train_input,val_input,train_output,val_output = train_test_split(data1['tweet'], data1['label'],random_state = 0)
  return train_input,val_input,train_output,val_output

**WORD2VEC**


In [0]:
def word2vec(train_input,val_input,data1,c):
   if(c == 1):
      tfidf_vect = TfidfVectorizer(analyzer='word')
      tfidf_vect.fit(data1['tweet'])
      xtrain = tfidf_vect.transform(train_input)
      valtrain = tfidf_vect.transform(val_input)
   elif(c == 2):
      tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
      tfidf_vect_ngram.fit(data1['tweet'])
      xtrain =  tfidf_vect_ngram.transform(train_input)
      valtrain =  tfidf_vect_ngram.transform(val_input)
   elif(c == 3):
      tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
      tfidf_vect_ngram_chars.fit(data1['tweet'])
      xtrain =  tfidf_vect_ngram_chars.transform(train_input) 
      valtrain =  tfidf_vect_ngram_chars.transform(val_input) 
   elif(c == 4):
      count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
      count_vect.fit(data1['tweet'])
      xtrain =  count_vect.transform(train_input)
      valtrain =  count_vect.transform(val_input)
   return xtrain,valtrain

In [0]:
def model_fitting(xtrain,train_output,model_name):
  if(model_name == 'NAVIE BAYES'):
    nb = naive_bayes.MultinomialNB()
    model1 = nb.fit(xtrain, train_output)
    #predictions = model1.predict(valtrain)
    #metrics.accuracy_score(predictions, val_output)
  elif(model_name == 'LogisticRegression'):
    lr = LogisticRegression()
    model1 = lr.fit(xtrain, train_output)
  elif(model_name == 'SVM'):
    svc = LinearSVC()
    model1 = svc.fit(xtrain, train_output)  
  elif(model_name == 'xgb'):
    xgb1 = xgb.XGBClassifier()
    model1 = xgb1.fit(xtrain,train_output)
    #predictions1 = model1.predict(valtrain)
  elif(model_name == 'RandomForest') :
    rf = RandomForestClassifier()
    model1 = rf.fit(xtrain, train_output)
    #predictions1 = model1.predict(valtrain)
    #print(metrics.accuracy_score(predictions1, val_output))
  return model1    

In [0]:
def model_prediction(model,valtrain,val_output):
  predictions = model.predict(valtrain)
  print(metrics.accuracy_score(predictions, val_output))

In [0]:
def training(data1,model_name,c):
  data1 = data_preparation(data1)
  train_input,val_input,train_output,val_output = data_split(data1)
  xtrain,valtrain = word2vec(train_input,val_input,data1,c)
  model1 = model_fitting(xtrain,train_output,model_name)
  #model1.save(model_name + c + '.h5')
  model_prediction(model1,valtrain,val_output)

In [0]:
def testing(data2,c,model_name):
  data2 = data_preparation(data2)
  if(model_name == 'NAVIE BAYES'):
    model1 = naive_bayes.MultinomialNB()
    model1 = load_model(model_name + c + '.h5')
    xtest = word2vec(train_input,data2,c)
    predictions = model1.predict(xtest)
  elif(model_name == 'LogisticRegression'):
    model1 = LogisticRegression()
    model1 = load_model(model_name + c + '.h5')
    xtest = word2vec(train_input,data2,c)
    predictions = model1.predict(xtest)
  elif(model_name == 'SVM'):
    model1 = LinearSVC()
    model1 = load_model(model_name + c + '.h5')
    xtest = word2vec(train_input,data2,c)
    predictions = model1.predict(xtest) 
  elif(model_name == 'xgb'):
    model1 = xgb.XGBClassifier()
    model1 = load_model(model_name + c + '.h5')
    xtest = word2vec(train_input,data2,c)
    predictions = model1.predict(xtest)
  elif(model_name == 'RandomForest') :
    model1 = RandomForestClassifier()
    model1 = load_model(model_name + c + '.h5')
    xtest = word2vec(train_input,data2,c)
    predictions = model1.predict(xtest)
  return predictions   

**NAVIE BAYES**

In [79]:
for i in range(1,5):
  training(data1,'NAVIE BAYES',i)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.8469696969696969


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.8171717171717172


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


0.8732323232323232


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.8808080808080808


**LOGISTIC REGRESSION**

In [80]:
for i in range(1,5):
  training(data1,'LogisticRegression',i)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.8555555555555555


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.7893939393939394


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


0.8666666666666667


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.8742424242424243


**SVM**

In [0]:
for i in range(1,5):
  training(data1,'SVM',i)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.8636363636363636


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


**XGboost**

In [81]:
for i in range(1,5):
  training(data1,'xgb',i)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.8262626262626263


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.76010101010101


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


0.854040404040404


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.8318181818181818


**random forest**

In [82]:
for i in range(1,5):
  training(data1,'RandomForest',i)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.8601010101010101


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.8212121212121212


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


0.8550505050505051


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.8757575757575757
